# SLAY Model 🌈

## BERT - Classification (emotion, category)

### Dependency

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm.auto import tqdm

### Dataset, Dataloader

In [ ]:
# Load the dataset
data = pd.read_csv('SLAYData_BERT_BART.csv')  # Ensure your CSV has columns: text, emotion, cause

# Define emotions and causes
emotions = ['Anxiety', 'Fatigue', 'Fear', 'Regret', 'Stress', 'Expectation', 'Worry']
causes = ['Academic', 'Employment/Career', 'Workplace', 'Family', 'Human Relationships', 'Financial Matters', 'Partner/Romantic Relationships']

# Encode labels
emotion_label_map = {label: idx for idx, label in enumerate(emotions)}
cause_label_map = {label: idx for idx, label in enumerate(causes)}

data['emotion_label'] = data['emotion'].map(emotion_label_map)
data['cause_label'] = data['cause'].map(cause_label_map)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_emotion = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotions))
model_cause = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(causes))
tokenizer_common = tokenizer

# Tokenize and encode sequences
inputs = tokenizer(data['text'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=512)
labels_emotion = torch.tensor(data['emotion_label'].values)
labels_cause = torch.tensor(data['cause_label'].values)

# Create dataset and dataloader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels_emotion, labels_cause)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Optimizer, Training

In [ ]:
# Set up optimizer
optimizer_emotion = AdamW(model_emotion.parameters(), lr=1e-6)
optimizer_cause = AdamW(model_cause.parameters(), lr=1e-6)

# Training function
def train(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids, attention_mask, labels = batch[:3]
        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, target_names):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, attention_mask, labels = batch[:3]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, preds)
    report = classification_report(true_labels, preds, target_names=target_names)
    return accuracy, report

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Train

In [ ]:
# Training loop for emotion
epochs = 3
for epoch in range(epochs):
    train_loss = train(model_emotion, train_dataloader, optimizer_emotion)
    val_accuracy, val_report = evaluate(model_emotion, val_dataloader, emotions)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}")
    print(val_report)

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1, Train Loss: 1.9272458775838217, Validation Accuracy: 0.3466666666666667
              precision    recall  f1-score   support

     Anxiety       0.00      0.00      0.00        49
     Fatigue       0.27      1.00      0.43        51
        Fear       0.51      1.00      0.67        45
      Regret       0.32      0.20      0.25        40
      Stress       0.00      0.00      0.00        37
 Expectation       0.00      0.00      0.00        38
       Worry       0.00      0.00      0.00        40

    accuracy                           0.35       300
   macro avg       0.16      0.31      0.19       300
weighted avg       0.17      0.35      0.21       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/75 [00:00<?, ?it/s]

...

Epoch 2, Train Loss: 1.637241063117981, Validation Accuracy: 0.4533333333333333
              precision    recall  f1-score   support

     Anxiety       0.00      0.00      0.00        49
     Fatigue       0.50      1.00      0.67        51
        Fear       0.00      0.00      0.00        45
      Regret       0.47      1.00      0.64        40
      Stress       0.00      0.00      0.00        37
 Expectation       1.00      0.13      0.23        38
       Worry       0.37      1.00      0.54        40

    accuracy                           0.45       300
   macro avg       0.33      0.45      0.30       300
weighted avg       0.32      0.45      0.30       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 3, Train Loss: 1.4506369845072429, Validation Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

     Anxiety       1.00      0.29      0.44        49
     Fatigue       0.89      1.00      0.94        51
        Fear       1.00      0.42      0.59        45
      Regret       0.61      1.00      0.75        40
      Stress       1.00      0.14      0.24        37
 Expectation       1.00      0.82      0.90        38
       Worry       0.37      1.00      0.54        40

    accuracy                           0.67       300
   macro avg       0.84      0.67      0.63       300
weighted avg       0.85      0.67      0.64       300



In [ ]:
model_cause = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(causes))
optimizer_cause = AdamW(model_cause.parameters(), lr=1e-6)

# Training loop for cause
epochs = 3
for epoch in range(epochs):
    train_loss = train(model_cause, train_dataloader, optimizer_cause)
    val_accuracy, val_report = evaluate(model_cause, val_dataloader, causes)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}")
    print(val_report)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1, Train Loss: 1.904726955095927, Validation Accuracy: 0.27666666666666667
                                precision    recall  f1-score   support

                      Academic       1.00      0.12      0.22        49
             Employment/Career       0.00      0.00      0.00        51
                     Workplace       1.00      0.51      0.68        45
                        Family       1.00      0.15      0.26        40
           Human Relationships       0.36      0.22      0.27        37
             Financial Matters       0.00      0.00      0.00        38
Partner/Romantic Relationships       0.16      1.00      0.28        40

                      accuracy                           0.28       300
                     macro avg       0.50      0.29      0.24       300
                  weighted avg       0.51      0.28      0.24       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 2, Train Loss: 1.7176605049769085, Validation Accuracy: 0.42
                                precision    recall  f1-score   support

                      Academic       0.43      1.00      0.60        49
             Employment/Career       0.00      0.00      0.00        51
                     Workplace       0.00      0.00      0.00        45
                        Family       1.00      0.15      0.26        40
           Human Relationships       0.00      0.00      0.00        37
             Financial Matters       0.66      0.82      0.73        38
Partner/Romantic Relationships       0.30      1.00      0.47        40

                      accuracy                           0.42       300
                     macro avg       0.34      0.42      0.29       300
                  weighted avg       0.33      0.42      0.29       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 3, Train Loss: 1.5516058524449667, Validation Accuracy: 0.8166666666666667
                                precision    recall  f1-score   support

                      Academic       0.70      1.00      0.82        49
             Employment/Career       1.00      0.59      0.74        51
                     Workplace       1.00      0.67      0.80        45
                        Family       1.00      0.78      0.87        40
           Human Relationships       1.00      0.73      0.84        37
             Financial Matters       0.79      1.00      0.88        38
Partner/Romantic Relationships       0.62      1.00      0.77        40

                      accuracy                           0.82       300
                     macro avg       0.87      0.82      0.82       300
                  weighted avg       0.87      0.82      0.82       300



### Predict

In [ ]:
# Prediction function
def predict_slay(text):
    model_emotion.eval()
    model_cause.eval()
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        emotion_outputs = model_emotion(inputs['input_ids'],  attention_mask=inputs['attention_mask'])
        cause_outputs = model_cause(inputs['input_ids'], attention_mask=inputs['attention_mask'])

        emotion_logits = emotion_outputs.logits
        cause_logits = cause_outputs.logits

        emotion_probs = torch.softmax(emotion_logits, dim=1).tolist()[0]
        cause_probs = torch.softmax(cause_logits, dim=1).tolist()[0]

        predicted_emotion_idx = torch.argmax(emotion_logits, dim=1).item()
        predicted_cause_idx = torch.argmax(cause_logits, dim=1).item()

        predicted_emotion = emotions[predicted_emotion_idx]
        predicted_cause = causes[predicted_cause_idx]

        emotion_word = emotions[predicted_emotion_idx].lower()
        cause_word = causes[predicted_cause_idx].lower().replace('_', ' ')

        result = f"It looks like you are feeling {emotion_word} from {cause_word} (confidence: {emotion_probs[predicted_emotion_idx]*100:.2f}%)"

    return {
        'predicted_emotion': predicted_emotion,
        'emotion_probs': dict(zip(emotions, emotion_probs)),
        'predicted_cause': predicted_cause,
        'cause_probs': dict(zip(causes, cause_probs)),
        'result': result
    }




In [ ]:
# Example prediction
text = "I am so tired because of the heavy project for the difficult coding class.”"
result = predict_slay(text)
print(result['result'])
print(f"Emotion Probabilities: {result['emotion_probs']}")
print(f"Cause Probabilities: {result['cause_probs']}")

It looks like you are feeling expectation from partner/romantic relationships (confidence: 20.37%)
Emotion Probabilities: {'Anxiety': 0.11678771674633026, 'Fatigue': 0.1928759664297104, 'Fear': 0.15604764223098755, 'Regret': 0.09992706775665283, 'Stress': 0.12947475910186768, 'Expectation': 0.20369918644428253, 'Worry': 0.10118760913610458}
Cause Probabilities: {'Academic': 0.1340220868587494, 'Employment/Career': 0.12937669456005096, 'Workplace': 0.10826577991247177, 'Family': 0.1461753249168396, 'Human Relationships': 0.170309916138649, 'Financial Matters': 0.14078880846500397, 'Partner/Romantic Relationships': 0.17106130719184875}


## BERT - Cause Finder(Extractor)

In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer

In [ ]:
class CauseExtractionModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased'):
        super(CauseExtractionModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(p=0.3)
        self.start_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.end_classifier = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        pooled_output = outputs[1]

        sequence_output = self.dropout(sequence_output)

        start_logits = self.start_classifier(sequence_output).squeeze(-1)
        end_logits = self.end_classifier(sequence_output).squeeze(-1)

        return start_logits, end_logits

tokenizer_ext = BertTokenizer.from_pretrained('bert-base-uncased')
model_ext = CauseExtractionModel()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast
import torch
import numpy as np

# Load data
file_path = 'SLAYData_BERT_BART.csv'
data = pd.read_csv(file_path)

# Split data into training and validation sets
train_texts, val_texts, train_causes, val_causes = train_test_split(
    data['text'], data['direct_word'], test_size=0.2, random_state=42
)

In [ ]:
# Dataset class
class CauseSpanDataset(torch.utils.data.Dataset):
    def __init__(self, texts, causes, tokenizer, max_len):
        self.texts = texts
        self.causes = causes
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        cause = self.causes.iloc[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_offsets_mapping=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        offsets = encoding['offset_mapping'][0]

        start_idx = text.find(cause)
        end_idx = start_idx + len(cause)

        start_positions = np.zeros(len(input_ids))
        end_positions = np.zeros(len(input_ids))

        for i, (start, end) in enumerate(offsets):
            if start <= start_idx < end:
                start_positions[i] = 1
            if start < end_idx <= end:
                end_positions[i] = 1

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(start_positions, dtype=torch.float),
            'end_positions': torch.tensor(end_positions, dtype=torch.float)
        }

# Create Datasets and DataLoaders
train_cause_dataset = CauseSpanDataset(train_texts, train_causes, tokenizer, max_len=128)
val_cause_dataset = CauseSpanDataset(val_texts, val_causes, tokenizer, max_len=128)

train_cause_loader = torch.utils.data.DataLoader(train_cause_dataset, batch_size=16, shuffle=True)
val_cause_loader = torch.utils.data.DataLoader(val_cause_dataset, batch_size=16, shuffle=False)


In [ ]:
# Model definition
class CauseExtractionModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased'):
        super(CauseExtractionModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(p=0.3)
        self.start_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.end_classifier = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask, return_dict=False)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        start_logits = self.start_classifier(sequence_output).squeeze(-1)
        end_logits = self.end_classifier(sequence_output).squeeze(-1)
        return start_logits, end_logits

# Initialize model and optimizer
model_ext = CauseExtractionModel()
optimizer_ext = AdamW(model_ext.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Training function
def train_cause(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']

        optimizer.zero_grad()
        start_logits, end_logits = model(input_ids, attention_mask)

        start_loss = nn.BCEWithLogitsLoss()(start_logits, start_positions)
        end_loss = nn.BCEWithLogitsLoss()(end_logits, end_positions)
        loss = (start_loss + end_loss) / 2

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    return avg_loss

def evaluate_cause(model, dataloader):
    model.eval()
    total_loss = 0
    all_start_preds, all_end_preds = [], []
    all_start_labels, all_end_labels = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            start_positions = batch['start_positions']
            end_positions = batch['end_positions']

            start_logits, end_logits = model(input_ids, attention_mask)

            start_loss = nn.BCEWithLogitsLoss()(start_logits, start_positions)
            end_loss = nn.BCEWithLogitsLoss()(end_logits, end_positions)
            loss = (start_loss + end_loss) / 2

            total_loss += loss.item()

            start_preds = (torch.sigmoid(start_logits) > 0.5).long()
            end_preds = (torch.sigmoid(end_logits) > 0.5).long()

            all_start_preds.extend(start_preds.cpu().numpy())
            all_end_preds.extend(end_preds.cpu().numpy())
            all_start_labels.extend(start_positions.cpu().numpy())
            all_end_labels.extend(end_positions.cpu().numpy())

    avg_loss = total_loss / len(dataloader)

    start_accuracy = accuracy_score(np.concatenate(all_start_labels), np.concatenate(all_start_preds))
    end_accuracy = accuracy_score(np.concatenate(all_end_labels), np.concatenate(all_end_preds))

    return avg_loss, start_accuracy, end_accuracy


In [ ]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    train_loss = train_cause(model_ext, train_cause_loader, optimizer_ext)
    val_loss, start_acc, end_acc = evaluate_cause(model_ext, val_cause_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    print(f"Start Position Accuracy: {start_acc:.4f}, End Position Accuracy: {end_acc:.4f}")

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1/1
Train Loss: 0.1036, Validation Loss: 0.0257
Start Position Accuracy: 0.9922, End Position Accuracy: 0.9922


In [ ]:
def predict_cause(model, text, tokenizer):
    model.eval()
    encoding = tokenizer(
        text,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_offsets_mapping=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    offsets = encoding['offset_mapping'][0]

    with torch.no_grad():
        start_logits, end_logits = model(input_ids, attention_mask)

    start_idx = torch.argmax(start_logits, dim=1).item()
    end_idx = torch.argmax(end_logits, dim=1).item()

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
    cause = tokenizer.convert_tokens_to_string(tokens[start_idx:end_idx + 1]).replace(" ##", "")

    return cause



Predicted Cause: nlp


In [ ]:
# Example prediction
text = "I am so tired because of my feelings for crush.”"
cause = predict_cause(model_ext, text, tokenizer)
print(f"Predicted Cause: {cause}")

Predicted Cause: feelings for


## Save SLAY BERT Model to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch

# Create a directory to save the model
os.makedirs('/content/drive/MyDrive/bert_models/emotion_model', exist_ok=True)
os.makedirs('/content/drive/MyDrive/bert_models/cause_model', exist_ok=True)
os.makedirs('/content/drive/MyDrive/bert_models/ext_model', exist_ok=True)

# Save emotion model
model_emotion.save_pretrained('/content/drive/MyDrive/bert_models/emotion_model')
tokenizer_common.save_pretrained('/content/drive/MyDrive/bert_models/emotion_model')

# Save cause model
model_cause.save_pretrained('/content/drive/MyDrive/bert_models/cause_model')
tokenizer_common.save_pretrained('/content/drive/MyDrive/bert_models/cause_model')

# Save extractor model (weights only)
torch.save(model_ext.state_dict(), '/content/drive/MyDrive/bert_models/ext_model/model_ext.pth')
tokenizer_ext.save_pretrained('/content/drive/MyDrive/bert_models/ext_model')


('/content/drive/MyDrive/bert_models/ext_model/tokenizer_config.json',
 '/content/drive/MyDrive/bert_models/ext_model/special_tokens_map.json',
 '/content/drive/MyDrive/bert_models/ext_model/vocab.txt',
 '/content/drive/MyDrive/bert_models/ext_model/added_tokens.json',
 '/content/drive/MyDrive/bert_models/ext_model/tokenizer.json')

# QUEEN Model 👸🏻

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

In [ ]:
import transformers
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

class PositiveTextGenerator:
    def __init__(self, model_id="meta-llama/Meta-Llama-3-8B-Instruct"):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True
        )

        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16
        )
        self.pipeline = transformers.pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer
        )

    def generate_text(self, messages):
        # Apply chat template to the messages
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Define terminators
        eos_token_id = self.tokenizer.eos_token_id
        terminators = [
            eos_token_id,
            self.tokenizer.convert_tokens_to_ids("")
        ]

        # Generate text
        outputs = self.pipeline(
            prompt,
            max_new_tokens=256,
            eos_token_id=eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )

        # Extract generated text
        generated_text = outputs[0]["generated_text"][len(prompt):]
        return generated_text

# Instantiate the generator
generator = PositiveTextGenerator()

# Example usage
messages = [
    {"role": "system", "content": "You will help envision a person with problems imagine a normal day without such problems. Add slangs like SLAY, YAS, QUEEN many times, and tweak sentences to add positive meanings and so on so that the person can resemble a day in which their problems no more exist. Emotions and Cause is mentioned in the First like with unique identifier #"},
    {"role": "user", "content": "{#anxiety from #romantic problems. I was worried that my crush will leave me if I told him about how I feel }"},
]

positive_text = generator.generate_text(messages)
print(positive_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Honey, let's get this day started and SLAY those anxiety vibes away!

Imagine waking up to a bright and sunny morning, feeling like the QUEEN of the world! You stretch your arms, and as you get out of bed, you notice a sweet text from your crush: "Good morning, beautiful! Can't wait to see you today!"

You take a deep breath, and as you get ready for the day, you can't help but feel a sense of YAS-ness! You're rocking that outfit, and your hair is looking FLAWLESS!

As you head out the door, you notice the birds singing, and the sun shining down on you. You feel a sense of calm wash over you, like a gentle breeze on a summer day. You're not worried about what your crush will think or say; you're just enjoying the moment!

You meet up with your crush, and as you chat, you realize that they're not only into you but genuinely care about your feelings. They listen attentively as you share your thoughts, and you feel a sense of relief wash over you. You're not alone, and you're not going to

# Load SLAY Model for Hosting

In [ ]:
import torch
from google.colab import drive
from transformers import BertTokenizer, BertForSequenceClassification

# Mount Google Drive
drive.mount('/content/drive')

# Load emotion model
emotion_model_path = '/content/drive/MyDrive/bert_models/emotion_model'
model_emotion = BertForSequenceClassification.from_pretrained(emotion_model_path)
tokenizer_emotion = BertTokenizer.from_pretrained(emotion_model_path)

# Load cause model
cause_model_path = '/content/drive/MyDrive/bert_models/cause_model'
model_cause = BertForSequenceClassification.from_pretrained(cause_model_path)
tokenizer_cause = BertTokenizer.from_pretrained(cause_model_path)

# Define the custom CauseExtractionModel class
class CauseExtractionModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased'):
        super(CauseExtractionModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(p=0.3)
        self.start_classifier = nn.Linear(self.bert.config.hidden_size, 1)
        self.end_classifier = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        start_logits = self.start_classifier(sequence_output).squeeze(-1)
        end_logits = self.end_classifier(sequence_output).squeeze(-1)
        return start_logits, end_logits

# Load the extractor model
ext_model_path = '/content/drive/MyDrive/bert_models/ext_model/model_ext.pth'
model_ext = CauseExtractionModel()
model_ext.load_state_dict(torch.load(ext_model_path))
model_ext.eval()

# Load tokenizer
tokenizer_ext = BertTokenizer.from_pretrained('/content/drive/MyDrive/bert_models/ext_model')

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Definition, Load and Test Predict Function (Copied)

In [ ]:
# Define emotions and causes
emotions = ['Anxiety', 'Fatigue', 'Fear', 'Regret', 'Stress', 'Expectation', 'Worry']
causes = ['Academic', 'Employment/Career', 'Workplace', 'Family', 'Human Relationships', 'Financial Matters', 'Partner/Romantic Relationships']

# Evaluation function
def evaluate(model, dataloader, target_names):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, attention_mask, labels = batch[:3]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, preds)
    report = classification_report(true_labels, preds, target_names=target_names)
    return accuracy, report

# Prediction function
def predict_slay(text):
    model_emotion.eval()
    model_cause.eval()
    model_ext.eval()
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        emotion_outputs = model_emotion(inputs['input_ids'],  attention_mask=inputs['attention_mask'])
        cause_outputs = model_cause(inputs['input_ids'], attention_mask=inputs['attention_mask'])
        ext_outputs = model_ext(inputs['input_ids'], attention_mask=inputs['attention_mask'])

        emotion_logits = emotion_outputs.logits
        cause_logits = cause_outputs.logits
        # ext_logits = ext_outputs.logits

        emotion_probs = torch.softmax(emotion_logits, dim=1).tolist()[0]
        cause_probs = torch.softmax(cause_logits, dim=1).tolist()[0]
        # ext_probs = torch.softmax(ext_logits, dim=1).tolist()[0]

        predicted_emotion_idx = torch.argmax(emotion_logits, dim=1).item()
        predicted_cause_idx = torch.argmax(cause_logits, dim=1).item()
        # predicted_ext_idx = torch.argmax(ext_logits, dim=1).item()

        predicted_emotion = emotions[predicted_emotion_idx]
        predicted_cause = causes[predicted_cause_idx]
        # predicted_ext = ext_probs[predicted_ext_idx]

        emotion_word = emotions[predicted_emotion_idx].lower()
        cause_word = causes[predicted_cause_idx].lower().replace('_', ' ')

        result = f"It looks like you are feeling {emotion_word} from {cause_word} (confidence: {emotion_probs[predicted_emotion_idx]*100:.2f}%)"

    return {
        'predicted_emotion': predicted_emotion,
        'emotion_probs': dict(zip(emotions, emotion_probs)),
        'predicted_cause': predicted_cause,
        'cause_probs': dict(zip(causes, cause_probs)),

        ##do something here!!
        'predicted_ext': text.split()[torch.argmax(ext_outputs[0], dim=1).item()-1: torch.argmax(ext_outputs[1], dim=1).item()+1],
        'result': result,
        'confidence': emotion_probs[predicted_emotion_idx]*100
    }

# Example prediction - test the model.
text = "I am stressed about my assignment."
result = predict_slay(text)
print(result['result'])
print(result['predicted_ext'])
print(f"Emotion Probabilities: {result['emotion_probs']}")
print(f"Cause Probabilities: {result['cause_probs']}")

It looks like you are feeling worry from partner/romantic relationships (confidence: 28.67%)
['assignment.']
Emotion Probabilities: {'Anxiety': 0.1649799346923828, 'Fatigue': 0.1437012404203415, 'Fear': 0.05455772206187248, 'Regret': 0.07293497771024704, 'Stress': 0.18004515767097473, 'Expectation': 0.09707650542259216, 'Worry': 0.286704421043396}
Cause Probabilities: {'Academic': 0.1267710030078888, 'Employment/Career': 0.15731224417686462, 'Workplace': 0.1656421720981598, 'Family': 0.08443014323711395, 'Human Relationships': 0.12901830673217773, 'Financial Matters': 0.12338539212942123, 'Partner/Romantic Relationships': 0.21344071626663208}


# Host the Model

## ngrok - Colab and Internet

In [ ]:
!pip install --upgrade Flask
!pip install ngrok pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.7 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token('MY_TOKEN')
public_url = ngrok.connect(addr="5000", hostname="MY_HOSTNAME")

print("Public URL:", public_url)


from flask import Flask, request, jsonify

app = Flask(__name__)

# Define the prediction function - SLAY Model
@app.route('/predictslay', methods=['POST'])
def slay():
    data = request.json
    text = data['text']
    result = predict_slay(text)
    print(text,'->', result['result'])
    return jsonify(result)

# Define the prediction function - QUEEN Model
@app.route('/predictqueen', methods=['POST'])
def queen():
    print('QUEEN')
    data = request.json
    text = data.get('text', '')
    print(text)
    if text:
        # Call the predict function with the provided text
        prompt = [{"role": "system", "content": text}]
        result = generator.generate_text(prompt)
    else:
        result = {'result': "No text provided"}
    print(result)
    # Return the result as JSON
    return jsonify({"result": result})

# Run the Flask app
if __name__ == '__main__':
    app.run(port=5000)

Public URL: NgrokTunnel: "https://obliging-relaxing-urchin.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jun/2024 05:16:40] "POST /predictslay HTTP/1.1" 200 -


 Good boi. -> It looks like you are feeling fatigue from partner/romantic relationships (confidence: 25.47%)


INFO:werkzeug:127.0.0.1 - - [18/Jun/2024 05:17:40] "POST /predictslay HTTP/1.1" 200 -


 -> It looks like you are feeling expectation from human relationships (confidence: 19.68%)


...

QUEEN
I was suffering from Academic Worry. I want to generate a daily journal of an ordinary that I no longer suffer from them. Tweak the following sentence:  I'm so stressed about my assignment.


INFO:werkzeug:127.0.0.1 - - [18/Jun/2024 05:29:44] "POST /predictqueen HTTP/1.1" 200 -


Here are a few options to help you reframe your thought and create a more positive daily journal entry:

1. "I'm feeling focused and confident about tackling my assignment today."
2. "I'm taking things one step at a time and breaking down my assignment into manageable tasks."
3. "I'm excited to learn and grow from this assignment, and I'm looking forward to the sense of accomplishment I'll feel when it's complete."
4. "I'm practicing self-care and taking a deep breath, reminding myself that I've got this and can handle whatever comes my way."
5. "I'm choosing to approach my assignment with a growth mindset, focusing on the opportunity to learn and improve rather than getting overwhelmed by stress."

Try to pick the one that resonates with you the most, or feel free to modify any of these sentences to fit your personal style and needs. Remember, the goal is to reframe your thinking and cultivate a more positive and empowering mindset.

Here's an example of what your daily journal entry 

INFO:werkzeug:127.0.0.1 - - [18/Jun/2024 05:30:44] "POST /predictqueen HTTP/1.1" 200 -


Here are a few options to help you reframe your sentence and cultivate a more positive mindset:

1. Today, I'm taking things one step at a time and focusing on making progress on my assignment.
2. I'm feeling a sense of calm and confidence as I work on my assignment, and I'm excited to see it come together.
3. My assignment is a challenge, but I'm up for it! I'm breaking it down into smaller tasks and tackling each one with ease.
4. I'm choosing to view my assignment as an opportunity to learn and grow, and I'm approaching it with an open and curious mind.
5. I've got this! I've been working hard and I'm feeling prepared to tackle my assignment and come out on top.

Remember, the key is to focus on the present moment and what you can control, rather than getting caught up in worries about the future or past. By reframing your thoughts and emotions, you can shift your mindset and feel more calm and confident.


## Should I merge two separate BERT Classification Model?

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, classification_report
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('SLAYData.csv')  # Ensure your CSV has columns: text, emotion, cause

# Define emotions and causes
emotions = ['Anxiety', 'Fatigue', 'Fear', 'Regret', 'Stress', 'Expectation', 'Worry']
causes = ['Academic', 'Employment/Career', 'Workplace', 'Family', 'Human Relationships', 'Financial Matters', 'Partner/Romantic Relationships']

# Encode labels
emotion_label_map = {label: idx for idx, label in enumerate(emotions)}
cause_label_map = {label: idx for idx, label in enumerate(causes)}

data['emotion_label'] = data['emotion'].map(emotion_label_map)
data['cause_label'] = data['cause'].map(cause_label_map)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences
inputs = tokenizer(data['text'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=512)
labels_emotion = torch.tensor(data['emotion_label'].values)
labels_cause = torch.tensor(data['cause_label'].values)

# Create dataset and dataloader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels_emotion, labels_cause)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define the multi-task model
class MultiTaskBERT(nn.Module):
    def __init__(self, model_name, num_emotions, num_causes):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier_emotion = nn.Linear(self.bert.config.hidden_size, num_emotions)
        self.classifier_cause = nn.Linear(self.bert.config.hidden_size, num_causes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        emotion_logits = self.classifier_emotion(pooled_output)
        cause_logits = self.classifier_cause(pooled_output)
        return emotion_logits, cause_logits

# Instantiate the model
model = MultiTaskBERT('bert-base-uncased', len(emotions), len(causes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids, attention_mask, labels_emotion, labels_cause = [item.to(device) for item in batch]
        optimizer.zero_grad()
        emotion_logits, cause_logits = model(input_ids, attention_mask)
        loss_emotion = criterion(emotion_logits, labels_emotion)
        loss_cause = criterion(cause_logits, labels_cause)
        loss = loss_emotion + loss_cause
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader):
    model.eval()
    emotion_preds, emotion_true = [], []
    cause_preds, cause_true = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, attention_mask, labels_emotion, labels_cause = [item.to(device) for item in batch]
            emotion_logits, cause_logits = model(input_ids, attention_mask)
            emotion_preds.extend(torch.argmax(emotion_logits, dim=1).tolist())
            emotion_true.extend(labels_emotion.tolist())
            cause_preds.extend(torch.argmax(cause_logits, dim=1).tolist())
            cause_true.extend(labels_cause.tolist())
    emotion_accuracy = accuracy_score(emotion_true, emotion_preds)
    cause_accuracy = accuracy_score(cause_true, cause_preds)
    emotion_report = classification_report(emotion_true, emotion_preds, target_names=emotions)
    cause_report = classification_report(cause_true, cause_preds, target_names=causes)
    return emotion_accuracy, cause_accuracy, emotion_report, cause_report

# Training and evaluation loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_dataloader, criterion, optimizer)
    print(f"Training loss: {train_loss:.4f}")
    emotion_accuracy, cause_accuracy, emotion_report, cause_report = evaluate(model, val_dataloader)
    print(f"Emotion accuracy: {emotion_accuracy:.4f}")
    print(f"Cause accuracy: {cause_accuracy:.4f}")
    print("Emotion classification report:")
    print(emotion_report)
    print("Cause classification report:")
    print(cause_report)

# Save the model and tokenizer
save_directory = '/content/drive/MyDrive/bert_models/slay_model'
os.makedirs(save_directory, exist_ok=True)

# Custom save_pretrained function
def save_pretrained_custom(model, tokenizer, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Save model weights
    model_path = os.path.join(save_directory, "pytorch_model.bin")
    torch.save(model.state_dict(), model_path)

    # Save model config
    config = model.bert.config
    config.save_pretrained(save_directory)

    # Save tokenizer
    tokenizer.save_pretrained(save_directory)

# Save model and tokenizer using the custom function
save_pretrained_custom(model, tokenizer, save_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

  0%|          | 0/50 [00:00<?, ?it/s]

Training loss: 2.1756


  0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Emotion accuracy: 0.1000
Cause accuracy: 1.0000
Emotion classification report:
              precision    recall  f1-score   support

     Anxiety       0.10      0.17      0.12        24
     Fatigue       0.00      0.00      0.00        34
        Fear       0.00      0.00      0.00        36
      Regret       0.15      0.15      0.15        34
      Stress       0.00      0.00      0.00        24
 Expectation       0.13      0.21      0.16        24
       Worry       0.11      0.25      0.15        24

    accuracy                           0.10       200
   macro avg       0.07      0.11      0.08       200
weighted avg       0.07      0.10      0.08       200

Cause classification report:
                                precision    recall  f1-score   support

                      Academic       1.00      1.00      1.00        30
             Employment/Career       1.00      1.00      1.00        34
                     Workplace       1.00      1.00      1.00        29
      

In [ ]:
# Prediction function
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        emotion_logits, cause_logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

        emotion_probs = torch.softmax(emotion_logits, dim=1).tolist()[0]
        cause_probs = torch.softmax(cause_logits, dim=1).tolist()[0]

        predicted_emotion_idx = torch.argmax(emotion_logits, dim=1).item()
        predicted_cause_idx = torch.argmax(cause_logits, dim=1).item()

        predicted_emotion = emotions[predicted_emotion_idx]
        predicted_cause = causes[predicted_cause_idx]

        emotion_word = emotions[predicted_emotion_idx].lower()
        cause_word = causes[predicted_cause_idx].lower().replace('_', ' ')

        result = f"It looks like you are feeling {emotion_word} from {cause_word} (confidence: {emotion_probs[predicted_emotion_idx]*100:.2f}%)"

    return {
        'predicted_emotion': predicted_emotion,
        'emotion_probs': dict(zip(emotions, emotion_probs)),
        'predicted_cause': predicted_cause,
        'cause_probs': dict(zip(causes, cause_probs)),
        'result': emotion_probs[predicted_emotion_idx]*100,
        'confidence': emotion_probs[predicted_emotion_idx]*100
    }


In [ ]:
predict("I have two assignments due tomorrow")